### 4.6.1 重新审视过拟合
当面对更多的特征而样本不足时，线性模型往往会过拟合。 相反，当给出更多样本而不是特征，通常线性模型不会过拟合。 不幸的是，线性模型泛化的可靠性是有代价的。 简单地说，线性模型没有考虑到特征之间的交互作用。 对于每个特征，线性模型必须指定正的或负的权重，而忽略其他特征。

泛化性和灵活性之间的这种基本权衡被描述为偏差-方差权衡（bias-variance tradeoff）。 线性模型有很高的偏差：它们只能表示一小类函数。 然而，这些模型的方差很低：它们在不同的随机数据样本上可以得出相似的结果。

深度神经网络位于偏差-方差谱的另一端。 与线性模型不同，神经网络并不局限于单独查看每个特征，而是学习特征之间的交互。 例如，神经网络可能推断“尼日利亚”和“西联汇款”一起出现在电子邮件中表示垃圾邮件， 但单独出现则不表示垃圾邮件。

即使我们有比特征多得多的样本，深度神经网络也有可能过拟合。 2017年，一组研究人员通过在随机标记的图像上训练深度网络。 这展示了神经网络的极大灵活性，因为人类很难将输入和随机标记的输出联系起来， 但通过随机梯度下降优化的神经网络可以完美地标记训练集中的每一幅图像。 想一想这意味着什么？ 假设标签是随机均匀分配的，并且有10个类别，那么分类器在测试数据上很难取得高于10%的精度， 那么这里的泛化差距就高达90%，如此严重的过拟合。

深度网络的泛化性质令人费解，而这种泛化性质的数学基础仍然是悬而未决的研究问题。 我们鼓励喜好研究理论的读者更深入地研究这个主题。 本节，我们将着重对实际工具的探究，这些工具倾向于改进深层网络的泛化性。

### 4.6.4 从零开始实现

In [1]:
import torch
from torch import nn
from d2l import torch as d2l


def dropout_layer(X, dropout):
    assert 0 <= dropout <= 1
    # 在本情况中，所有元素都被丢弃
    if dropout == 1:
        return torch.zeros_like(X)
    # 在本情况中，所有元素都被保留
    if dropout == 0:
        return X
    mask = (torch.rand(X.shape) > dropout).float()
    return mask * X / (1.0 - dropout)

In [2]:
X= torch.arange(16, dtype = torch.float32).reshape((2, 8))
print(X)
print(dropout_layer(X, 0.))
print(dropout_layer(X, 0.5))
print(dropout_layer(X, 1.))

tensor([[ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.],
        [ 8.,  9., 10., 11., 12., 13., 14., 15.]])
tensor([[ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.],
        [ 8.,  9., 10., 11., 12., 13., 14., 15.]])
tensor([[ 0.,  2.,  4.,  6.,  8., 10., 12.,  0.],
        [ 0.,  0., 20., 22.,  0.,  0.,  0.,  0.]])
tensor([[0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.]])


In [3]:
num_inputs, num_outputs, num_hiddens1, num_hiddens2 = 784, 10, 256, 256

In [4]:
dropout1, dropout2 = 0.2, 0.5

class Net(nn.Module):
    def __init__(self, num_inputs, num_outputs, num_hiddens1, num_hiddens2,
                 is_training = True):
        super(Net, self).__init__()
        self.num_inputs = num_inputs
        self.training = is_training
        self.lin1 = nn.Linear(num_inputs, num_hiddens1)
        self.lin2 = nn.Linear(num_hiddens1, num_hiddens2)
        self.lin3 = nn.Linear(num_hiddens2, num_outputs)
        self.relu = nn.ReLU()

    def forward(self, X):
        H1 = self.relu(self.lin1(X.reshape((-1, self.num_inputs))))
        # 只有在训练模型时才使用dropout
        if self.training == True:
            # 在第一个全连接层之后添加一个dropout层
            H1 = dropout_layer(H1, dropout1)
        H2 = self.relu(self.lin2(H1))
        if self.training == True:
            # 在第二个全连接层之后添加一个dropout层
            H2 = dropout_layer(H2, dropout2)
        out = self.lin3(H2)
        return out


net = Net(num_inputs, num_outputs, num_hiddens1, num_hiddens2)

In [5]:
num_epochs, lr, batch_size = 10, 0.5, 256
loss = nn.CrossEntropyLoss(reduction='none')
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size)
trainer = torch.optim.SGD(net.parameters(), lr=lr)
d2l.train_ch3(net, train_iter, test_iter, loss, num_epochs, trainer)

### 4.6.5 简洁实现

In [6]:
net = nn.Sequential(nn.Flatten(),
        nn.Linear(784, 256),
        nn.ReLU(),
        # 在第一个全连接层之后添加一个dropout层
        nn.Dropout(dropout1),
        nn.Linear(256, 256),
        nn.ReLU(),
        # 在第二个全连接层之后添加一个dropout层
        nn.Dropout(dropout2),
        nn.Linear(256, 10))

def init_weights(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, std=0.01)

net.apply(init_weights);

In [7]:
trainer = torch.optim.SGD(net.parameters(), lr=lr)
d2l.train_ch3(net, train_iter, test_iter, loss, num_epochs, trainer)

### Practice
### 1. 如果更改第一层和第二层的暂停法概率，会发生什么情况？具体地说，如果交换这两个层，会发生什么情况？设计一个实验来回答这些问题，定量描述该结果，并总结性的结论。

**实验设计**：
- 设置两层神经网络，并分别设定第一层和第二层的暂停概率（Dropout Rate）。
- 两组实验：
  1. 第一层暂停率为 $p_1$，第二层暂停率为 $p_2$；
  2. 交换两层的暂停率，即第一层为 $p_2$，第二层为 $p_1$。
- 记录训练和测试的误差，并观察两种设置的性能。

**结果总结**：
1. 通常情况下，较高的暂停率对靠近输入层的影响更大，因为输入层特征受损可能导致后续层学习不稳定。
2. 当交换两层暂停率时，模型性能可能有所下降，尤其在第一层暂停率过高时。
3. 两个层的暂停概率需要根据具体问题调优，不能单纯地交换。

---

### 2. 增加训练轮数，并将使用暂停法和不使用暂停法时获得的结果进行比较。

**分析方法**：
- 在相同训练轮数下，比较使用和不使用暂停法的训练和测试误差。
- 逐渐增加训练轮数，观察模型的收敛趋势。

**结果总结**：
1. **使用暂停法**：
   - 初期的训练误差较高，但随着训练轮数增加，模型对测试集的性能逐渐提高，表现为更好的泛化能力。
2. **不使用暂停法**：
   - 训练误差迅速下降，但测试误差可能较高，表明模型容易过拟合。
3. **结论**：
   - 暂停法通过随机丢弃部分神经元，强制模型在有限的特征下学习，增加了泛化能力，尤其在训练轮数较多时表现出优势。

---

### 3. 当前用或不用暂停法时，每个隐藏层中激活值的方差是多少？给出一个曲线图，以显示这两个模型中每个隐藏层中激活值的方差是如何随时间变化的。

**分析方法**：
- 记录每个隐藏层激活值的方差，分别在使用和不使用暂停法的情况下比较。
- 绘制激活值方差随训练轮数变化的曲线。

**结果总结**：
- **使用暂停法**：激活值的方差较平稳，表明模型的特征表达更加稳定。
- **不使用暂停法**：激活值的方差可能较大，且波动较为明显，尤其在深层网络中更为显著。

---

### 4. 为什么在测试时通常不使用暂停法？

在测试阶段，模型需要评估性能，暂停法会随机丢弃神经元，导致模型输出不稳定。因此，测试时通常不使用暂停法，而是将训练期间的丢弃率通过缩放权重来反映。在测试阶段：
- 每个神经元的输出会乘以 $1 - p$（其中 $p$ 是暂停概率）。
- 这样可以模拟训练阶段的期望输出，保持一致性。

---

### 5. 以本节中的模型为例，比较使用暂停法和权重衰减的效果。如果同时使用暂停法和权重衰减，会发生什么情况？结果是累加的吗？收敛是否减少（或者说更糟）？它们互相抵消了吗？

**分析**：
- **单独使用暂停法**：通过随机丢弃神经元来减少过拟合，增强泛化能力。
- **单独使用权重衰减**：通过在损失函数中加入权重的正则化项，抑制过大的权重值。
- **同时使用**：
  1. 两种方法可以互补，权重衰减控制参数规模，暂停法增强模型的鲁棒性。
  2. 模型可能需要更长的训练时间才能收敛。
  3. 如果参数选择不当（如权重衰减系数过大），可能会导致相互抵消，结果变得更差。

---

### 6. 如果我们将暂停法应用到权重矩阵的各个权重，而不是激活值，会发生什么？

如果将暂停法应用于权重矩阵的各个权重：
- **效果**：
  1. 权重矩阵中部分权重被随机置零，导致权重更新不完整。
  2. 可能引入额外的梯度噪声，训练过程更加不稳定。
- **潜在问题**：
  - 权重矩阵中的零值可能阻断信息流，尤其对深层网络影响更大。

这种方法通常不如对激活值进行暂停法有效，但可以通过实验比较其性能。

---

### 7. 发明一种用于在每一层注入随机噪声的技术，该技术不同于标准的暂停法技术。尝试开发一种在Fashion-MNIST数据集（对于固定架构）上性能优于暂停法的方法。

**新技术：激活值扰动法（Activation Perturbation）**：
- 在每一层的激活值上添加小幅随机噪声：
  $$
  h' = h + \epsilon, \quad \epsilon \sim \mathcal{N}(0, \sigma^2)
  $$
  其中，$\epsilon$ 是服从零均值正态分布的随机噪声。

**执行步骤**：
1. 在每一前向传播中，对每层的激活值添加随机噪声。
2. 调整噪声强度 $\sigma$，确保噪声不会过大以破坏特征表达。

**实验验证**：
- 在Fashion-MNIST数据集上测试该方法，与标准暂停法（Dropout）进行对比。
- 观察训练误差和测试误差。

**结果预期**：
- 激活值扰动法通过增加随机性，防止网络过拟合，同时避免了暂停法中神经元完全丢弃的问题。
- 若调整得当，可能在某些任务上优于标准暂停法。